In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Variable Funding Leg + Equity or Cash Instrument

Demonstrates creation and pricing of a funding leg with 
variable notional and constructing a related position in
an stock or underlying instrument. This construct can be used
to represent the mechanics of a total return or equity swap.

Attributes
----------
funding leg swap
instruments
aggregation
market data store
results store
quotes
"""

toggle_code("Hide docstring")

### Pricing a Funding Leg with an Equity Position

In [2]:
# Import LUSID
import lusid
import lusid.models as lm
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
    format_quotes_response,
)

# Import notebook specific utilities
from utilities.instrument_utils import (
    add_utc_to_df,
    valuation_response_to_df,
    upsert_instrument,
    create_property,
    equity_swap_transaction
)

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import pandas as pd
from lusidjam.refreshing_token import RefreshingToken
import os

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lusid.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.8411.0


In [3]:
# Setup the apis we'll use in this notebook:
aggregation_api = api_factory.build(lusid.AggregationApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)

# Set the scope we'll use in this notebook:
scope = "variable-funding-leg"

# 1. Setup

## 1.1 Create Instruments

We begin by defining the details of the funding leg with the following characteristics:

- Inception Date: 2021-04-05
- Maturity Date: 2022-04-05
- Currency: USD
- Initial Notional: 0
- Index: USD 3M Libor
- Spread: 25 bp


Notice that the initial notional is set to zero, as we will be booking this later on in the transactions. For accruals, the funding leg will store the notional history and accrue accordingly.

In [4]:
# Set swap details for start date and maturity
start_date = datetime(2021, 4, 5, 0, 0, tzinfo=pytz.utc)

effective_delta = 1

# Create a 1y 'USD' Funding leg specifications
effective_at = start_date
end_date = datetime(2022, 4, 5, 0, 0, tzinfo=pytz.utc)
currency = "USD"
initial_notional = 0
spread = 0.0025
leg_identifier="FundingLeg001"
leg_name = "Variable Funding Leg"

In [5]:
# Define the flow conventions of the floating leg
floating_leg_convention = lm.FlowConventions(
    currency=currency,
    payment_frequency="3M",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0
)

# Define the flow conventions of the underlying index
floating_leg_idx_convention = lm.IndexConvention(
    currency=currency,
    code="LIBOR",
    payment_tenor="3M",
    fixing_reference="USD3M",
    publication_day_lag=0,
    day_count_convention="Act360"
)

# Set the leg definition and side
floating_leg_definition = lm.LegDefinition(
    conventions=floating_leg_convention,
    index_convention = floating_leg_idx_convention,
    notional_exchange_type = "None",
    pay_receive = "Pay",
    rate_or_spread = spread,
    stub_type = "Back"
)

# Define the funding leg
funding_leg = lm.FundingLeg(
    start_date=start_date,
    maturity_date=end_date,
    notional=initial_notional,
    leg_definition=floating_leg_definition,
    instrument_type="FundingLeg"
)

# Upsert the instrument to LUSID
upsert_instrument(api_factory=api_factory, name=leg_name, identifier=leg_identifier, definition=funding_leg)

Instrument Variable Funding Leg was successfully upserted into LUSID
Instrument created with LUID:LUID_IJ2ZEC61


Next we will define and upsert the equity that will be paired with the funding leg to construct the swap. In this case we will be using an **AMZN** stock as the example.

In [6]:
# Set the details of the stock
equity_name = "Amazon"
equity_identifier = "AMZN"
dom_ccy = "USD"

In [7]:
# Define the instrument
equity = lm.SimpleInstrument(
    instrument_type="SimpleInstrument",
    dom_ccy=dom_ccy,
    asset_class="Equities",
    simple_instrument_type="Equity",
)

# Upsert the instrument
upsert_instrument(api_factory=api_factory, name=equity_name, identifier=equity_identifier, definition=equity)

Instrument Amazon was successfully upserted into LUSID
Instrument created with LUID:LUID_0000CZRI


## 1.2 Setting up Market Data

In [8]:
# Scope used to store our market data
market_data_scope = "FBN"
# The market data supplier
market_supplier = 'Lusid'

## 1.3 Equity Quotes

In [9]:
# Read quotes and adjust date objects
df = pd.read_csv("data/amzn_prices.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
add_utc_to_df(df)
df.head()

,Date,Price,Ticker
0,2020-12-15 00:00:00+00:00,"3,165.12",AMZN
1,2020-12-16 00:00:00+00:00,"3,240.96",AMZN
2,2020-12-17 00:00:00+00:00,"3,236.08",AMZN
3,2020-12-18 00:00:00+00:00,"3,201.65",AMZN
4,2020-12-21 00:00:00+00:00,"3,206.18",AMZN


In [10]:
# Create a quotes mapping for Lusid Python Tools
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "Date",
    "quote_id.quote_series_id.field": "$mid",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "Ticker",
    "metric_value.value": "Price",
    "metric_value.unit": "$USD",
}

result = load_from_data_frame(
    api_factory = api_factory,
    scope=market_data_scope,
    data_frame=df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,253,0,0


## 1.4 Funding Leg Rates

For valuation, we will also require some simple market data, or quotes. In this particular case, we will need USD3M Libor resets for our swaps inception date and for the valuation date (`effective_at`).

In [11]:
fix_date_1 = start_date
fixing = 0.0173

reset_series_id = lm.QuoteSeriesId(
    provider="Lusid",
    price_source=None,
    instrument_id="USD3M",
    instrument_id_type="RIC",
    quote_type="Rate",
    field="mid"
)


QuoteId = lm.QuoteId(
    quote_series_id=reset_series_id,
    effective_at=fix_date_1.isoformat()
)
USD3mQuoteRequest = lm.UpsertQuoteRequest(
    quote_id=QuoteId,
    metric_value=lm.MetricValue(
        value=fixing, unit="rate"),
    lineage="USD libor"
)

quotes_request = {fix_date_1.isoformat(): USD3mQuoteRequest}

response = quotes_api.upsert_quotes(scope=market_data_scope,
                      request_body=quotes_request)

if response.failed == {}:
    print(f"Quotes successfully upserted into LUSID. {len(quotes_request)} quotes upserted.")
else:
    print(f"An error occurred with the above upsert_quotes call for fixing, see response:")
    print(response)

Quotes successfully upserted into LUSID. 1 quotes upserted.


Likewise, we will need quotes for our underlying index, in order to determine the equity leg accruals. For this example we upload 1 year's worth of _SPX Index_ quotes, which will cover the full life cycle of our swap. For larger datasets, we can use [`lusidtools`](https://github.com/finbourne/lusid-python-tools/wiki) which allow us to use a mapping schema to read our data using a pandas dataframe.

## 1.5 Create Portfolio and sub-holding key

We continue using the `load_from_data_frame` tool, but in this case use it to first build out a portfolio under the scope setup below.

We will also have to create the `LinkId` property, which is to be used as a sub-holding key for the portfolio that links the equity and funding leg component together.

In [12]:
create_property(api_factory=api_factory, name="Linking ID", domain="Transaction", scope="common", code="LinkId", data_type="string")

Property Transaction/common/LinkId already exists


In [13]:
# Setup scope and code for the portfolio
trading_scope = "Finbourne-Examples"
trading_code = "FundingLegWithEquity"

# Set sub-holding keys
sub_holding_keys = ["LinkId"]

# Setup a dataframe from which we will creat the portfolio
data = {'portfolio_code':  [trading_code],
        'portfolio_name': [trading_code],
       }

portfolio_df= pd.DataFrame(data, columns=['portfolio_code','portfolio_name'])

# Create a mapping schema for the portfolio
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$USD",
    },
    "optional": {"created": "$2019-01-01T00:00:00+00:00"},
}

In [14]:
# A portfolio can be loaded using a dataframe by setting file_type to "portfolios"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=trading_scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=sub_holding_keys,
    sub_holding_keys_scope="common"
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

,success,failed
0,1,0


# 1.6 Book Transactions

With the portfolio in LUSID, we can book a transaction against our swap. For this we can use the [TransactionPortfoliosApi](https://www.lusid.com/docs/api#operation/UpsertTransactions).

For uploading the transactions we will use a utility function `equity_swap_transaction`, that is defined in a parallel file under utilities directory.

The transaction will package the two separate equity and funding leg transactions into one, with the direction being set by the directional indicator `S` or `L`. In LUSID this simply defines whether or not the transaction will book notional with a `StockIn` or `StockOut` transaction code.

Additionally, we pass a `linkId` which can be used the 2 components together, as will be shown later when valuing the instrument.

In [15]:
txn_df = pd.read_csv("data/equity_swap_trades.csv")
txn_df["trade_date"] = pd.to_datetime(txn_df["trade_date"], dayfirst=True)
add_utc_to_df(txn_df)
txn_df

,transaction_id,notional,equity_identifier,funding_leg_identifier,transaction_currency,direction,trade_date,linking_id,identifier_type
0,TXN001,200000,AMZN,FundingLeg001,USD,S,2021-04-05 00:00:00+00:00,EQ-SW-001-AMZN,ClientInternal
1,TXN002,100000,AMZN,FundingLeg001,USD,S,2021-04-07 00:00:00+00:00,EQ-SW-001-AMZN,ClientInternal
2,TXN003,100000,AMZN,FundingLeg001,USD,L,2021-04-09 00:00:00+00:00,EQ-SW-001-AMZN,ClientInternal
3,TXN004,200000,AMZN,FundingLeg001,USD,L,2021-04-14 00:00:00+00:00,EQ-SW-001-AMZN,ClientInternal


In [16]:
link_id_property_key = "Transaction/common/LinkId"

for i, row in txn_df.iterrows():
    equity_swap_transaction(
        api_factory=api_factory,
        portfolio_scope=trading_scope,
        portfolio_code=trading_code,
        notional=row["notional"],
        equity_identifier=row["equity_identifier"],
        funding_leg_identifier=row["funding_leg_identifier"],
        transaction_currency=row["transaction_currency"],
        direction=row["direction"],
        trade_date=row["trade_date"].isoformat(),
        transaction_id=row["transaction_id"],
        linking_id=row["linking_id"],
        linking_id_property=link_id_property_key,
        
    )

Equity and Funding Leg transactions loaded into portfolio with scope Finbourne-Examples and code FundingLegWithEquity.
Equity and Funding Leg transactions loaded into portfolio with scope Finbourne-Examples and code FundingLegWithEquity.
Equity and Funding Leg transactions loaded into portfolio with scope Finbourne-Examples and code FundingLegWithEquity.
Equity and Funding Leg transactions loaded into portfolio with scope Finbourne-Examples and code FundingLegWithEquity.


# 2. Valuation
For interacting with the valuation engine, we will need a configuration recipe, which are a set of steps that define how a valuation is to be carried out. Generally, this is part of LUSID configuration that requires a one time setup, below we will be focusing on the _FundingLeg_ instrument.

## 2.1 Configure the valuation recipe
We begin by defining the pricing context under which valuation will be done, this allows to select the model under which we can value the instrument. In our case we have selected _"ConstantTimeValueOfMoney"_, which will use a deterministic pricer to calculate our portfolio PV/Accrual.

In [17]:
# Define instrument model config
instrument_models = {
    "FundingLeg": "ConstantTimeValueOfMoney",
    "SimpleInstrument": "SimpleStatic"
}

def create_pricing_context(market_supplier: str, instrument_model_config: dict):
    
    model_rules = [
        lm.VendorModelRule(
                supplier=market_supplier,
                model_name=model,
                instrument_type=instrument_type,
                parameters="{}",
            )
        for instrument_type, model in instrument_model_config.items()
    ]
    
    return lm.PricingContext(
        model_rules = model_rules,
         options=lm.PricingOptions(
            model_selection=lm.ModelSelection(
                        library=market_supplier,
                        model="SimpleStatic"
                    )
         )
    )

# Create the pricing context for the recipe
pricing_context = create_pricing_context(market_supplier, instrument_models)

Next, we need to define the market data context that will be used by the valuation engine. In this case we will set the market data provider as 'Lusid' and use our `market_data_scope`, as set previously.

Moreover, within our market_rules, we need to specify the keys by which the valuation engine will resolve the discount curves and USD Libor fixings.

In [18]:
def create_market_context(market_data_scope, supplier):
    return lm.MarketContext(
        # Set rules for where we should resolve our rates and equity data.
        market_rules=[
            lm.MarketDataKeyRule(
                key='Quote.ClientInternal.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),
             lm.MarketDataKeyRule(
                key='Quote.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Rate',
                field='mid',
                quote_interval="2D.0D"),
            lm.MarketDataKeyRule(
                key='Quote.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),

        ],
        # Control default options for resolving market data.
        options=lm.MarketOptions(
            default_supplier=supplier,
            default_instrument_code_type="ClientInternal",
            default_scope=market_data_scope)
     )

    return market_context

# In our case simply default to the LUSID market_supplier.
market_context = create_market_context(market_data_scope, market_supplier)

With both the pricing model and market data defintions setup in the above models, we can finalize the pricing recipe as shown below using the [`ConfigurationRecipe`](https://github.com/finbourne/lusid-sdk-python-preview/blob/master/sdk/lusid/models/configuration_recipe.py) model. We will again set the market data scopes, as well as give our recipe a name/code and brief description.

In [19]:
def create_pricing_recipe(scope, market_context, pricing_context):

    return lm.ConfigurationRecipe(
        scope=scope,
        code="FundingLeg",
        description="Funding leg pricing recipe",
        market=market_context,
        pricing=pricing_context
    )

# Create the funding leg pricing recipe using our market_data_scope
pricing_recipe = create_pricing_recipe(market_data_scope, market_context, pricing_context)

In [20]:
def upsert_recipe(recipe):
    recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
    
    return configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = recipe_request)

# Upsert the previously created recipe to be used in valuation
response = upsert_recipe(pricing_recipe)
print(f"Recipe upserted at {response.value}")

Recipe upserted at 2022-01-05 14:40:03.909901+00:00


## 2.2 Aggregation
With the recipes and configurations set for our swap book, we can query the valuation engine for accruals and market values of the instruments. This can be done using the [`AggregationApi`](https://www.lusid.com/docs/api#tag/Aggregation), as shown below.

In [21]:
def aggregate_pricing(portfolio_scope, portfolio_code, recipe_scope, recipe_code, metrics, effective_at, effective_from=None, group_by=[]):
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope=recipe_scope,
            code=recipe_code
        ),
        metrics=metrics,
            portfolio_entity_ids=[
                lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
            ],
            valuation_schedule=lm.ValuationSchedule(
                effective_from=effective_from,
                effective_at=effective_at
            ),
        sort=[
            lm.OrderBySpec(key='Analytic/default/ValuationDate',
            sort_order='Ascending')
        ],
        group_by=group_by
        )
    return aggregation_api.get_valuation(
    valuation_request= valuation_request
    )

In [22]:
# Create the metrics list for the desired data
metrics = [
        lm.AggregateSpec(key='Valuation/PV',
                             op='Value'),
        lm.AggregateSpec(key='Instrument/default/Name',
                             op='Value'),
        lm.AggregateSpec(key='Holding/default/Units',
                             op='Value'),
        lm.AggregateSpec(key='Valuation/Accrued',
                             op='Value'),
        lm.AggregateSpec(key='Analytic/default/ValuationDate',
                             op='Value'),
        ]
# Store results
results = aggregate_pricing(trading_scope, trading_code, market_data_scope, "FundingLeg", metrics, (start_date + timedelta(days=8)).isoformat(), start_date)

As seen below, we have successfully generated a series of valuations for the swap including the position PVs and accruals for the funding leg. We can also notice how the daily accrual changes as we increase or decrease the notional against the funding leg.

In [23]:
# Display PVs
valuations = valuation_response_to_df(results)
valuations['Analytic/default/ValuationDate'] = valuations['Analytic/default/ValuationDate'].apply(lambda x : parse(x).strftime('%d/%m/%Y'))
valuations.set_index('Analytic/default/ValuationDate', inplace=True)
display(valuations)

,Holding/default/Units,Valuation/PV,Valuation/Accrued,Instrument/default/Name
Analytic/default/ValuationDate,,,,
05/04/2021,"-200,000.00","1,381.56",0.00,Variable Funding Leg
05/04/2021,"-200,000.00","-645,346,000.00",0.00,Amazon
06/04/2021,"-200,000.00","1,381.56",-11.00,Variable Funding Leg
06/04/2021,"-200,000.00","-644,764,000.00",0.00,Amazon
07/04/2021,"-300,000.00","2,061.33",-22.00,Variable Funding Leg
07/04/2021,"-300,000.00","-983,817,000.00",0.00,Amazon
08/04/2021,"-300,000.00","2,061.33",-38.50,Variable Funding Leg
08/04/2021,"-300,000.00","-989,790,000.00",0.00,Amazon
09/04/2021,"-200,000.00","1,392.56",-55.00,Variable Funding Leg


Similarly, we can also group the valuations by the previously created `linkId`, so that the equity swap package gets valued as a single line for each date.

This is illustrated below, notice we group by both the sub-holding key and the valuation date in our request.

In [24]:
# Create a new metrics list, changing the op to 'Sum'
metrics = [
        lm.AggregateSpec(key='Valuation/PV',
                             op='Sum'),
        lm.AggregateSpec(key='Transaction/common/LinkId',
                             op='Value'),
        lm.AggregateSpec(key='Valuation/Accrued',
                             op='Sum'),
        lm.AggregateSpec(key='Analytic/default/ValuationDate',
                             op='Value'),
        ]
# Store results
results = aggregate_pricing(trading_scope, trading_code, market_data_scope, "FundingLeg", metrics, (start_date + timedelta(days=8)).isoformat(), start_date, group_by=['Transaction/common/LinkId', 'Analytic/default/ValuationDate'])

In [25]:
# Display PVs
valuations = valuation_response_to_df(results)
valuations['Analytic/default/ValuationDate'] = valuations['Analytic/default/ValuationDate'].apply(lambda x : parse(x).strftime('%d/%m/%Y'))
valuations.set_index('Analytic/default/ValuationDate', inplace=True)
display(valuations)

,Transaction/common/LinkId,Sum(Valuation/Accrued),Sum(Valuation/PV)
Analytic/default/ValuationDate,,,
05/04/2021,EQ-SW-001-AMZN,0.00,"-645,344,618.44"
06/04/2021,EQ-SW-001-AMZN,-11.00,"-644,762,618.44"
07/04/2021,EQ-SW-001-AMZN,-22.00,"-983,814,938.67"
08/04/2021,EQ-SW-001-AMZN,-38.50,"-989,787,938.67"
09/04/2021,EQ-SW-001-AMZN,-55.00,"-674,438,607.44"
12/04/2021,EQ-SW-001-AMZN,-88.00,"-675,876,607.44"
13/04/2021,EQ-SW-001-AMZN,-99.00,"-679,998,607.44"
